In [14]:

import os
import shutil
import argparse
import traceback
from tqdm import tqdm
from typing import List, Dict, Tuple, Optional

from concurrent.futures import ThreadPoolExecutor, as_completed

import fiftyone as fo

from tator_tools.yolo_dataset import YOLODataset
from tator_tools.download_query import QueryDownloader
from tator_tools.train_model import ModelTrainer
from tator_tools.fiftyone_clustering import FiftyOneDatasetViewer

import tator

import cv2
import numpy as np
import pandas as pd

import torch
from ultralytics import YOLO
from ultralytics import RTDETR

from yolo_tiler import YoloTiler, TileConfig

# Custom DataDownloader (for getting select frames from media)

In [4]:
class DataDownloader:
    def __init__(self, api_token: str, project_id: int, media_ids: List[int], 
                 frame_ids_dict: Dict[int, List[int]], output_dir: str, 
                 max_workers: int = 10, max_retries: int = 10):
        """
        Initialize the DataDownloader with multiple media IDs and their corresponding frames.

        :param api_token: Tator API token for authentication
        :param project_id: Project ID in Tator
        :param media_ids: List of media IDs to process
        :param frame_ids_dict: Dictionary mapping media IDs to their frame IDs to download
        :param output_dir: Output directory for downloaded frames
        :param max_workers: Maximum number of concurrent download threads
        :param max_retries: Maximum number of retries for failed downloads
        """
        self.project_id = project_id
        self.media_ids = media_ids
        self.frames_dict = frame_ids_dict
        self.output_dir = output_dir
        self.max_workers = max_workers
        self.max_retries = max_retries
        
        # Create a single API instance for all operations
        self.api = self._authenticate(api_token)
        
        # Set up directories
        self._setup_directories()
        
        # Cache for media objects
        self.media_cache = {}
        
        # Output data
        self.output_data = None

    @staticmethod
    def _authenticate(api_token: str):
        """
        Authenticate with the Tator API.

        :param api_token: API token for authentication
        :return: Authenticated API instance
        """
        try:
            api = tator.get_api(host='https://cloud.tator.io', token=api_token)
            return api
        except Exception as e:
            raise Exception(f"ERROR: Could not authenticate with provided API Token\n{e}")

    def _setup_directories(self):
        """
        Create necessary directories for frame storage.
        """
        os.makedirs(f"{self.output_dir}/frames", exist_ok=True)

    def _get_media(self, media_id: int):
        """
        Get media object with caching to avoid redundant API calls.
        
        :param media_id: Media ID to retrieve
        :return: Media object
        """
        if media_id not in self.media_cache:
            self.media_cache[media_id] = self.api.get_media(id=int(media_id))
        return self.media_cache[media_id]

    def download_frame(self, params: tuple) -> Tuple[int, int, Optional[str]]:
        """
        Download a single frame for a given media with retry logic.

        :param params: Tuple containing (media_id, frame_id)
        :return: Tuple of (media_id, frame_id, frame_path or None if failed)
        """
        media_id, frame_id = params
        media = self._get_media(media_id)
        
        # Use absolute path for frame_path
        frame_path = os.path.abspath(f"{self.output_dir}/frames/{str(media_id)}_{str(frame_id)}.jpg")
        
        # Use absolute path for lock_path
        lock_path = f"{frame_path}.lock"
        
        # Rest of the method remains the same as before
        if os.path.exists(frame_path):
            return media_id, frame_id, frame_path
            
        if os.path.exists(lock_path):
            if os.path.getmtime(lock_path) < time.time() - 300:
                try:
                    os.remove(lock_path)
                except:
                    pass
            else:
                for _ in range(60):
                    time.sleep(1)
                    if os.path.exists(frame_path):
                        return media_id, frame_id, frame_path
                    if not os.path.exists(lock_path):
                        break
                
        try:
            with open(lock_path, 'w') as f:
                f.write(str(os.getpid()))
        except:
            time.sleep(1)
            if os.path.exists(frame_path):
                return media_id, frame_id, frame_path
        
        for attempt in range(self.max_retries):
            try:
                temp = self.api.get_frame(
                    id=media.id,
                    tile=f"{media.width}x{media.height}",
                    force_scale="1024x768",  # TODO remove hardcoding
                    frames=[int(frame_id)]
                )
                shutil.move(temp, frame_path)
                
                try:
                    os.remove(lock_path)
                except:
                    pass
                    
                return media_id, frame_id, frame_path
                
            except Exception as e:
                error_msg = f"Error downloading frame {frame_id} for media {media_id}: {e}"
                if attempt < self.max_retries - 1:
                    print(f"{error_msg}, retrying...")
                    time.sleep(2 ** attempt)
                else:
                    print(f"{error_msg}, giving up.")
        
        try:
            os.remove(lock_path)
        except:
            pass
            
        return media_id, frame_id, None

    def download_data(self) -> Dict[int, List[str]]:
        """
        Download frames for all media IDs using a single thread pool.

        :return: Dictionary mapping media IDs to lists of frame paths
        """
        # Prepare all download tasks
        all_tasks = []
        for media_id in self.media_ids:
            frames = self.frames_dict[media_id]
            for frame_id in frames:
                all_tasks.append((media_id, frame_id))
        
        results_dict = {media_id: [] for media_id in self.media_ids}
        
        # Use a single thread pool for all downloads
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            futures = {
                executor.submit(self.download_frame, task): task 
                for task in all_tasks
            }
            
            with tqdm(total=len(all_tasks), desc="Downloading frames") as pbar:
                for future in as_completed(futures):
                    media_id, frame_id, frame_path = future.result()
                    if frame_path:  # If download was successful
                        results_dict[media_id].append(frame_path)
                    pbar.update(1)
        
        self.output_data = results_dict


In [5]:
api_token = os.getenv("TATOR_TOKEN")
project_id = 155

output_dir="../Data/NCICS/Madeline_Data/"

# Read in CSV file

In [21]:
# Read and preprocess the data
df = pd.read_csv("../Data/NCICS/MadelineIs_All_Annotations_20241113.csv")

# Only drop rows where TatorMediaID OR TatorFrame have NA values
df = df.dropna(subset=['TatorMediaID', 'TatorFrame'])

# Convert the columns to integers after removing NA values
df['TatorMediaID'] = df['TatorMediaID'].astype(int)
df['TatorFrame'] = df['TatorFrame'].astype(int)

# Create dictionary mapping media IDs to their frame lists
media_ids = df['TatorMediaID'].unique().tolist()
frame_ids_dict = {media_id: df[df['TatorMediaID'] == media_id]['TatorFrame'].tolist() for media_id in media_ids}

# Download frames from CSV

In [ ]:
# Initialize downloader with multiple media IDs
downloader = DataDownloader(
    api_token=api_token,
    project_id=project_id,
    media_ids=media_ids,
    frame_ids_dict=frame_ids_dict,
    output_dir=output_dir,
    max_workers=10,
    max_retries=10,
)

# Download all frames for all media IDs
downloader.download_data()

In [25]:
frame_paths_dict = downloader.output_data

In [26]:
# Create a new dataframe with the paths
output_df = []
for media_id, group in df.groupby('TatorMediaID'):
    # Get the frame paths for this media ID
    media_frame_paths = frame_paths_dict[media_id]
    
    # Create a mapping of frame number to path
    frame_to_path = {
        int(path.split('_')[-1].replace('.jpg', '')): path 
        for path in media_frame_paths
    }
    
    # Add paths to the group
    group = group.copy()
    group['Image_Path'] = group['TatorFrame'].map(frame_to_path)
    output_df.append(group)

# Combine all groups back into a single dataframe
final_df = pd.concat(output_df, ignore_index=True)

In [ ]:
final_df[['TatorMediaID', 'TatorFrame', 'Image_Path', 'Sclass', 'Ssubclass', 'Sgroup']].head(3)

In [28]:
# Create and output updated dataframe with the paths
final_df.to_csv("../Data/NCICS/Madeline_Data/MadelineIs_Modified.csv", index=False)

# Download Unlabeled AUV Data

In [4]:
# Search string comes from Tator's Data Metadata Export utility
search_string = "eyJtZXRob2QiOiJBTkQiLCJvcGVyYXRpb25zIjpbeyJhdHRyaWJ1dGUiOiJNaXNzaW9uTmFtZSIsIm9wZXJhdGlvbiI6Imljb250YWlucyIsImludmVyc2UiOmZhbHNlLCJ2YWx1ZSI6Ik1hZGVsaW5lIn0seyJtZXRob2QiOiJPUiIsIm9wZXJhdGlvbnMiOlt7ImF0dHJpYnV0ZSI6IiR0eXBlIiwib3BlcmF0aW9uIjoiZXEiLCJpbnZlcnNlIjpmYWxzZSwidmFsdWUiOjMzMX1dfV19"

# Demo for downloading labeled data
frac = 0.01

dataset_name = "Unlabeled_AUV_Data"
output_dir = "../Data/NCICS/"

In [ ]:
# Create a downloader for the labeled data
downloader = DatasetDownloader(api_token,
                               project_id=project_id,
                               search_string=search_string,
                               frac=frac,
                               output_dir=output_dir,
                               dataset_name=dataset_name,
                               label_field="",
                               download_width=1024)

In [ ]:
# Download the labeled data
downloader.download_data()

In [11]:
df = downloader.as_dataframe()  # .as_dict()

In [12]:
df.to_csv("../Data/NCICS/Unlabeled_AUV_Data/Unlabeled_AUV_Data.csv", index=False)

# Make a YOLO Image Classification Dataset

In [2]:
# Get the labeled data, subset
labeled_df = pd.read_csv("../Data/NCICS/Madeline_Data/MadelineIs_Modified.csv")
labeled_df = labeled_df[['Path', 'Sclass', 'Ssubclass', 'Sgroup']]

df = labeled_df.copy()
df['image_path'] = df['Path']
df['image_name'] = df['Path'].apply(lambda x: os.path.basename(x))
df['label'] = df['Ssubclass']

# Fill with nulls
df['x'] = np.nan
df['y'] = np.nan
df['width'] = np.nan
df['height'] = np.nan
df['polygon'] = np.nan

# Show class distribution
df['label'].value_counts()

label
Gravel Substrate    219
Muddy Substrate     209
Mixed Gravels       177
Sandy Substrate     134
Bedrock              81
Trace Gravels        61
Unclassifiable       28
Name: count, dtype: int64

In [3]:
# Set parameters
output_dir = "../Data/NCICS/Madeline_Data/"
dataset_name = "YOLO_Classification_Dataset"

train_ratio = 0.8
test_ratio = 0.1

task = 'classify' # 'classify', 'detect', 'segment'

In [4]:
# Create and process dataset
dataset = YOLODataset(
    data=df,
    output_dir=output_dir,
    dataset_name=dataset_name,
    train_ratio=train_ratio,
    test_ratio=test_ratio,
    task=task,
    format_class_names=False, 
)

In [5]:
# Process the dataset
dataset.process_dataset(move_images=False)  # Makes a copy of the images instead of moving them

Processing YOLO dataset with 909 annotations...
Dataset split: 727 train, 92 valid, 90 test images
Creating classification dataset with 7 classes
  Bedrock: train=67, val=9, test=5
  Gravel Substrate: train=175, val=26, test=18
  Mixed Gravels: train=140, val=18, test=19
  Muddy Substrate: train=166, val=17, test=26
  Sandy Substrate: train=107, val=12, test=15
  Trace Gravels: train=51, val=7, test=3
  Unclassifiable: train=21, val=3, test=4


Copying images:   0%|          | 0/909 [00:00<?, ?it/s]

Dataset created at e:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset
Classes: ['Bedrock', 'Gravel Substrate', 'Mixed Gravels', 'Muddy Substrate', 'Sandy Substrate', 'Trace Gravels', 'Unclassifiable']


Rendering Examples:   0%|          | 0/10 [00:00<?, ?it/s]

Rendered 10 examples to e:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset\examples


In [7]:
dataset.output_dir

'e:\\tator-tools\\Data\\NCICS\\Madeline_Data'

# Tile YOLO Datset (Optional)

In [11]:
src = dataset.dataset_dir                                           # Source YOLO dataset directory
dst = f"{dataset.output_dir}\\YOLO_Classification_Dataset_Tiled"    # Output directory for tiled dataset

config = TileConfig(
    # Size of each tile (width, height). Can be:
    # - Single integer for square tiles: slice_wh=640
    # - Tuple for rectangular tiles: slice_wh=(640, 480)
    slice_wh=(768, 512),

    # Overlap between adjacent tiles. Can be:
    # - Single float (0-1) for uniform overlap percentage: overlap_wh=0.1
    # - Tuple of floats for different overlap in each dimension: overlap_wh=(0.1, 0.1)
    # - Single integer for pixel overlap: overlap_wh=64
    # - Tuple of integers for different pixel overlaps: overlap_wh=(64, 48)
    overlap_wh=(0.2, 0.2),

    # Input image file extension to process
    input_ext=".jpg",

    # Output image file extension to save (default: same as input_ext)
    output_ext=None,

    # Type of YOLO annotations to process:
    # - "image_classification": Standard YOLO format (class)
    # - "object_detection": Standard YOLO format (class, x, y, width, height)
    # - "instance_segmentation": YOLO segmentation format (class, x1, y1, x2, y2, ...)
    annotation_type="image_classification",

    # Include negative samples (tiles without any instances)
    include_negative_samples=True
)

tiler = YoloTiler(
    source=src,
    target=dst,
    config=config,
    num_viz_samples=15,                     # Number of samples to visualize
    show_processing_status=True,            # Show the progress of the tiling process
)

In [12]:
tiler.run()

2025-03-10 17:35:33,323 - YoloTiler - INFO - Found 727 images in train/ directory
2025-03-10 17:35:33,323 - YoloTiler - INFO - Found 727 images in train/ directory
2025-03-10 17:35:33,325 - YoloTiler - INFO - Found 727 label files in train/ directory
2025-03-10 17:35:33,325 - YoloTiler - INFO - Found 727 label files in train/ directory
2025-03-10 17:35:33,326 - YoloTiler - INFO - Processing e:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset\train\Bedrock\14666551_2522.jpg
2025-03-10 17:35:33,326 - YoloTiler - INFO - Processing e:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset\train\Bedrock\14666551_2522.jpg
train: Tile: 100%|██████████| 4/4 [00:00<00:00, 68.96tiles/s]
2025-03-10 17:35:33,405 - YoloTiler - INFO - Processing e:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset\train\Bedrock\14666558_1713.jpg
2025-03-10 17:35:33,405 - YoloTiler - INFO - Processing e:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset\train\Be

# Train a YOLO model

In [15]:
training_data = f"../Data/NCICS/Madeline_Data/YOLO_Classification_Dataset_Tiled/"

# Initialize the trainer with the required parameters
trainer = ModelTrainer(
    training_data=training_data,
    weights="yolov8m-cls.pt",
    output_dir=f"{training_data}/results",
    name="yolov8m",
    task='classify',
    epochs=100,
    patience=10,
    half=True,
    imgsz=640,
    single_cls=False,
    plots=True,
    batch=0.5,
)

100%|██████████| 32.7M/32.7M [00:00<00:00, 202MB/s]


In [16]:
# Train the model
trainer.train_model()

New https://pypi.org/project/ultralytics/8.3.86 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.10.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 16384MiB)
engine\trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=../Data/NCICS/Madeline_Data/YOLO_Classification_Dataset_Tiled/, epochs=100, time=None, patience=10, batch=0.5, imgsz=640, save=True, save_period=10, cache=False, device=0, workers=8, project=../Data/NCICS/Madeline_Data/YOLO_Classification_Dataset_Tiled//results, name=yolov8m, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

100%|██████████| 6.25M/6.25M [00:00<00:00, 134MB/s]


AMP: checks passed 
AutoBatch: Computing optimal batch size for imgsz=640 at 50.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 16.00G total, 0.25G reserved, 0.16G allocated, 15.59G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    15781303       41.89         0.470         41.34         55.34        (1, 3, 640, 640)                  (1, 7)
    15781303       83.78         0.696         18.67            35        (2, 3, 640, 640)                  (2, 7)
    15781303       167.6         1.231            26         38.34        (4, 3, 640, 640)                  (4, 7)
    15781303       335.1         2.391            40         51.34        (8, 3, 640, 640)                  (8, 7)
    15781303       670.2         4.603         79.67         93.01       (16, 3, 640, 640)                 (16, 7)
AutoBatch: Using batch-size 27 for CUDA:0 8.07G/16.00G (50%) 


train: Scanning E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\train... 2908 images, 0 corrupt: 100%|██████████| 2908/2908 [00:04<00:00, 717.97it/s]


train: New cache created: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\train.cache


...

                   all      0.783          1
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 7, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



17 epochs completed in 0.233 hours.
Optimizer stripped from ..\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\results\yolov8m\weights\last.pt, 31.7MB
Optimizer stripped from ..\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\results\yolov8m\weights\best.pt, 31.7MB

Validating ..\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\results\yolov8m\weights\best.pt...
Ultralytics 8.3.0  Python-3.10.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 16384MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,771,623 parameters, 0 gradients, 41.6 GFLOPs
train: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\train... found 2908 images in 7 classes  
val: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\val... found 368 images in 7 classes  
test: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\test... found 360 images in 7 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:02<00:00,  3.09it/s]


                   all      0.812      0.997
Speed: 1.2ms preprocess, 4.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to ..\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\results\yolov8m
Results saved to ..\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\results\yolov8m
Training completed.


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000188B815A5F0>
curves: []
curves_results: []
fitness: 0.904891312122345
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8125, 'metrics/accuracy_top5': 0.9972826242446899, 'fitness': 0.904891312122345}
save_dir: WindowsPath('../Data/NCICS/Madeline_Data/YOLO_Classification_Dataset_Tiled/results/yolov8m')
speed: {'preprocess': 1.160446716391522, 'inference': 4.750288051107655, 'loss': 0.0, 'postprocess': 0.002718490103016729}
task: 'classify'
top1: 0.8125
top5: 0.9972826242446899

In [17]:
# Evaluate on the model (if test data is available)
trainer.evaluate_model()

Ultralytics 8.3.0  Python-3.10.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 16384MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,771,623 parameters, 0 gradients, 41.6 GFLOPs
train: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\train... found 2908 images in 7 classes  
val: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\val... found 368 images in 7 classes  
test: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\test... found 360 images in 7 classes  


test: Scanning E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\test... 360 images, 0 corrupt: 100%|██████████| 360/360 [00:00<00:00, 959.93it/s]


test: New cache created: E:\tator-tools\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\test.cache


               classes   top1_acc   top5_acc: 100%|██████████| 14/14 [00:03<00:00,  3.51it/s]


                   all        0.8      0.997
Speed: 1.0ms preprocess, 5.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to ..\Data\NCICS\Madeline_Data\YOLO_Classification_Dataset_Tiled\results\yolov8m2


# Clustering

In [18]:
# Get the unlabeled data, subset, conform
unlabeled_df = pd.read_csv("../Data/NCICS/Unlabeled_AUV_Data/Unlabeled_AUV_Data.csv")
unlabeled_df = unlabeled_df[['image_path']]
unlabeled_df['Path'] = unlabeled_df['image_path']

# No labels, set as Unlabeled
unlabeled_df['Sclass'] = "Unlabeled"
unlabeled_df['Ssubclass'] = "Unlabeled"
unlabeled_df['Sgroup'] = "Unlabeled"

# Drop the image_path column
unlabeled_df.drop(columns=['image_path'], inplace=True)

unlabeled_df.head(3)

,Path,Sclass,Ssubclass,Sgroup
0,e:\tator-tools\Data\NCICS\Unlabeled_AUV_Data\i...,Unlabeled,Unlabeled,Unlabeled
1,e:\tator-tools\Data\NCICS\Unlabeled_AUV_Data\i...,Unlabeled,Unlabeled,Unlabeled
2,e:\tator-tools\Data\NCICS\Unlabeled_AUV_Data\i...,Unlabeled,Unlabeled,Unlabeled


In [19]:
# Combine the labeled and unlabeled data
combined_df = pd.concat([labeled_df, unlabeled_df], ignore_index=True)

# Perform QA / QC such that sClass, sSubclass, sGroup are not empty
# Replace any empty values or NaN values with "Unlabeled"
combined_df['Sclass'] = combined_df['Sclass'].fillna("Unlabeled").replace("", "Unlabeled")
combined_df['Ssubclass'] = combined_df['Ssubclass'].fillna("Unlabeled").replace("", "Unlabeled")
combined_df['Sgroup'] = combined_df['Sgroup'].fillna("Unlabeled").replace("", "Unlabeled")

combined_df.sample(3)

,Path,Sclass,Ssubclass,Sgroup
161,e:\tator-tools\Data\NCICS\Madeline_Data\frames...,Fine Unconsolidated Mineral Substrate,Sandy Substrate,Sand
932,e:\tator-tools\Data\NCICS\Unlabeled_AUV_Data\i...,Unlabeled,Unlabeled,Unlabeled
416,e:\tator-tools\Data\NCICS\Madeline_Data\frames...,Coarse Unconsolidated Mineral Substrate,Mixed Gravels,Gravel Mixes


In [26]:
embeddings = None

if True:
    # Calculate custom embeddings
    model_weights = "E:\\tator-tools\\Data\\NCICS\\Madeline_Data\\YOLO_Classification_Dataset_Tiled\\results\\yolov8m\\weights\\best.pt"
    # Load the model
    model = YOLO(model_weights)
    # Get the image size
    imgsz = model.__dict__['overrides']['imgsz']

    # Get the device
    device ='cuda' if torch.cuda.is_available() else 'cpu'
    print(f"NOTE: Using device {device}")

    # Run a blank image through the model to load the weights
    _ = model(np.zeros((imgsz, imgsz, 3), dtype=np.uint8), device=device) 

    embeddings_list = []

    # Use the length of combined_df as the total for tqdm
    total_items = len(combined_df)
    for path in tqdm(combined_df['Path'].tolist(), total=total_items, desc="Calculating embeddings"):
        embeddings = model.embed(path, imgsz=imgsz, stream=False, device=device, verbose=False)
        embeddings_list.append(embeddings[0].cpu().numpy())
        
    embeddings = np.array(embeddings_list)
    embeddings.shape

    torch.cuda.empty_cache()  

NOTE: Using device cuda

0: 640x640 Muddy Substrate 0.31, Sandy Substrate 0.19, Unclassifiable 0.14, Mixed Gravels 0.10, Trace Gravels 0.10, 100.0ms
Speed: 15.0ms preprocess, 100.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Calculating embeddings: 100%|██████████| 1179/1179 [01:28<00:00, 13.38it/s]


In [27]:
# Initialize the viewer with the path to the directory containing images
viewer = FiftyOneDatasetViewer(dataframe=combined_df,
                               image_path_column='Path',
                               feature_columns=['Sclass', 'Ssubclass', 'Sgroup'],
                               nickname='MadelineIs',
                               custom_embeddings=embeddings,  # Pass the embeddings, or None
                               clustering_method='umap',      # umap, pca, tsne
                               num_dims=2)                    # Number of dimensions for UMAP (2 or 3)

In [28]:
# Process the dataset to create the FiftyOne dataset and generate the UMAP visualization
viewer.process_dataset()

Overwriting existing dataset: MadelineIs


Processing images: 100%|██████████| 1179/1179 [00:35<00:00, 32.82it/s]


 100% |███████████████| 1179/1179 [757.9ms elapsed, 0s remaining, 1.6K samples/s]      
Computing embeddings...
Using provided custom embeddings
Computing UMAP visualization...
Generating visualization...


UMAP( verbose=True)
Mon Mar 10 18:06:21 2025 Construct fuzzy simplicial set
Mon Mar 10 18:06:22 2025 Finding Nearest Neighbors
Mon Mar 10 18:06:29 2025 Finished Nearest Neighbor Search
Mon Mar 10 18:06:32 2025 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Mon Mar 10 18:06:35 2025 Finished embedding


In [29]:
# Launch the FiftyOne app
try:
    session = fo.launch_app(viewer.dataset)
except:
    # Weird behavior in notebook
    session = fo.launch_app(viewer.dataset)

Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port
